# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [113]:
pd.set_option('max_rows', None)

In [21]:
nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html" )[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

#display(cities.head())

['Mo']

In [95]:
# Funções complementares exercício 1.
'''nhl_df['Metropolitan area'] = np.nan
    for index, name in enumerate(nhl_df['team']):
        pattern = name.split(' ')[-1]
        search_field = clean_cities()
        
        for i, row in enumerate(search_field['NHL']):
            if re.search(pattern, row):
                nhl_df.loc[index,'Metropolitan area'] = search_field.loc[i, 'Metropolitan area']'''

# Add zeros to percentages.
def add_zero(s):
    return float('0'+ s)

# Correcting team names (stage 1)
def correct_team_name1(s):
    pattern = '\[note \d+\]'
    return re.sub(pattern = pattern , repl  = '', string = s)

# Correcting rivalries names
def correct_team_name2(s):
    pattern = '(?P<first_name>[0-9A-Z][a-z]+)(?P<last_name>[A-Z][a-z]+)'
    if re.search(pattern, s):
        first_name = re.findall(pattern, s)[0][0]
        last_name = re.findall(pattern, s)[0][1]
        return first_name + '-' + last_name
    else:
        return s
    
# Cleaning the cities DF.
def clean_cities():
    global cities
    pattern = '(?P<name>[A-z.,\d -]+)(?P<note>\[note \d+\])'
    
    # Correcting the 'Population' column name and its data type
    cities.rename(columns = {'Population (2016 est.)[8]': 'Population'}, inplace = True)
    cities['Population'] = cities['Population'].astype('int')
    
    # Ajusting the teams names in the DF.
    big4 = ['NFL', 'MLB', 'NBA', 'NHL']
    for big in big4:
        cities[big] = (cities[big]
                    .apply(correct_team_name1)
                    .apply(correct_team_name2)) 
  
    # Returning the new DF
    return cities
    

In [97]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html" )[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# Cleaning the 'nhl_df' DF
def clean_nhl_df():
    global nhl_df
    
    # Cleaning the divisions rows
    pattern1 = '[A-z]+ (?P<division>Division)'
    for i, team in enumerate(nhl_df['team']):
        if re.search(pattern1, team):
            nhl_df.drop(index = i, inplace = True)
            
    # Removing the '*' from the teams names.
    pattern2 = '(?P<name>[A-z ]+)(?P<star>\*)'
    for i, team in zip(nhl_df.index, nhl_df['team']):
        find = re.findall(pattern2, team)
        if len(find) != 0:
            nhl_df.loc[i, 'team'] = find[0][0]
            
    # Adding a zero in front of the values indicating percentage.
    nhl_df['PTS%'] = nhl_df['PTS%'].apply(add_zero)
    nhl_df['RPt%'] = nhl_df['RPt%'].apply(add_zero)
            
    # Correcting the wrong data types.
    int_columns = ['GP', 'W', 'L', 'OL', 'PTS', 'PTS%', 'GF', 'GA', 'RPt%', 'ROW']
    float_columns = ['SRS', 'SOS']
    
    nhl_df[int_columns] = nhl_df[int_columns].astype('int')
    nhl_df[float_columns] = nhl_df[float_columns].astype('float')
    
    # Now, adding a 'Win/Loss' ratio column.
    nhl_df['Win/Loss'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])
    
    # Getting the metropolitan region for every team.
    nhl_df['Metropolitan area'] = np.nan
    
    for index, name in zip(nhl_df.index, nhl_df['team']):
        pattern = name.split(' ')[-1] 
        search_field = clean_cities()
        
        for i, row in enumerate(search_field['NHL']):
            if re.search(pattern, row):
                nhl_df.loc[index,'Metropolitan area'] = search_field.loc[i, 'Metropolitan area']
    
    # Finally, returning a pandas Series with the Metropolitan area name along with its teams Win/Loss ratio.
    return nhl_df[nhl_df['year'] == 2018].groupby('Metropolitan area')['Win/Loss'].mean()


def nhl_correlation(): 
    df = pd.merge(clean_cities(), clean_nhl_df(), on = 'Metropolitan area')
    
    population_by_region = list(df['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(df['Win/Loss'])# pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

nhl_correlation()

(-0.04027318804589655, 0.8387661496942489)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [133]:
# DROPAR AS COLUNAS COM OS NOMES DE DIVISÃO
# CORRIGIR OS NOMES DOS TIMES
# TRANSFORMAR COLUNAS NUMÉRICAS EM NÚMEROS.
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv", na_values = '—')
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def clean_nba_df():
    global nba_df
    nba_df = nba_df[nba_df['year'] == 2018]
    
    # Correcting the data types from the numeric columns
    int_columns = ['W','L', 'year']
    float_columns = ['W/L%', 'GB', 'PS/G', 'PA/G', 'SRS']
    
    nba_df[int_columns] = nba_df[int_columns].astype('int')
    nba_df[float_columns] = nba_df[float_columns].astype('float')
    return nba_df#.dtypes

def nba_correlation():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)

clean_nba_df()

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League
0,Toronto Raptors* (1),59,23,0.720,NaN,111.7,103.9,7.29,2018,NBA
1,Boston Celtics* (2),55,27,0.671,4.0,104.0,100.4,3.23,2018,NBA
2,Philadelphia 76ers* (3),52,30,0.634,7.0,109.8,105.3,4.30,2018,NBA
3,Cleveland Cavaliers* (4),50,32,0.610,9.0,110.9,109.9,0.59,2018,NBA
4,Indiana Pacers* (5),48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA
5,Miami Heat* (6),44,38,0.537,15.0,103.4,102.9,0.15,2018,NBA
6,Milwaukee Bucks* (7),44,38,0.537,15.0,106.5,106.8,-0.45,2018,NBA
7,Washington Wizards* (8),43,39,0.524,16.0,106.6,106.0,0.53,2018,NBA
8,Detroit Pistons (9),39,43,0.476,20.0,103.8,103.9,-0.26,2018,NBA
9,Charlotte Hornets (10),36,46,0.439,23.0,108.2,108.0,0.07,2018,NBA


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [105]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)


## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values